In [281]:
from pyscf.lib import krylov
from pyscf import gto, scf
import scipy
import inspect
import pyscf.qmmm
import pyscf.dft
import numpy as np
angstrom = 1 / 0.52917721067
from matplotlib import pyplot as plt
from functools import reduce
from numpy.linalg import inv 
from pyscf.scf import cphf 

def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords(), deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [282]:
mol=gto.Mole(atom="H 0 0 0 ;H 0 0 2.", unit="Bohr", basis= "sto-3g")
mf=scf.RHF(mol)
mf.scf()
dL=.1
mf1=fc(mf,[dL,-dL])
mf1.scf()
mf2=fc(mf,[-dL,dL])
mf2.scf()
dV=DeltaV(mol,[dL,-dL])
h2=mf.get_veff()
dh2=(mf1.get_veff()-mf2.get_veff())/2
J,K=mf.get_jk()
g_ijkl=mf.mol.intor('int2e_sph')
g_h2=g_ijkl-np.swapaxes(g_ijkl,1,2)/2
nocc= mol.nelec[0]


C=mf.mo_coeff
nmo=C.shape[0]
nvir=nmo-nocc
S=mf.get_ovlp()
e=mf.mo_energy
E=np.diag(e)
O=np.diag(mf.mo_occ)
C1=mf1.mo_coeff
P=mf.make_rdm1()
P1=mf1.make_rdm1()
P2=mf2.make_rdm1() 
dP=(P1-P2)/2
dP2=(P1-2*P+P2)/dL**2
dC=(abs(C1)*C/abs(C)-C)
dV_mo=C.T@dV@C

converged SCF energy = -1.04917090198581
converged SCF energy = -1.06995377385842


Initialize <pyscf.gto.mole.Mole object at 0x7feb103a8470> in <pyscf.scf.hf.RHF object at 0x7feb103a8668>


converged SCF energy = -1.06995377385842


In [283]:
def test(u_):
    ut=np.zeros((nmo,nmo))
    ut[nocc:,:nocc]=u_.reshape(nvir,nocc)
    ut[:nocc,nocc:]=-u_.reshape(nvir,nocc).T
    dP_app=C@(ut@O-O@ut)@C.T    
    print(np.linalg.norm(dP_app-dP))
    print(np.linalg.norm(dP))
    print(np.linalg.norm(dP_app))

In [284]:
nmo,nocc,nvir

(2, 1, 1)

In [285]:
print(U_0)

[[ 2.90549984e-17 -6.36855833e-02  9.33170824e-18 -5.08273736e-02
  -2.11212132e-02  3.20978999e-17]
 [ 6.36855833e-02 -2.42910985e-17  5.20971529e-02 -4.16726069e-17
   8.06869771e-18  1.46532364e-02]
 [-1.58124069e-17 -5.20971529e-02 -1.05793416e-17  1.55452391e-01
   1.93098118e-02 -5.05612630e-17]
 [ 5.08273736e-02  1.18199917e-17 -1.55452391e-01  5.53990589e-17
   8.32873306e-17  2.94187529e-02]
 [ 2.11212132e-02 -4.59328442e-18 -1.93098118e-02 -7.54070774e-17
   5.74897298e-16  1.07541470e+00]
 [-2.94250095e-17 -1.46532364e-02  4.89789727e-17 -2.94187529e-02
  -1.07541470e+00 -5.70319181e-16]]


In [286]:
U_0f

array([ 6.36855833e-02, -1.58124069e-17,  5.08273736e-02,  2.11212132e-02,
       -2.94250095e-17])

In [287]:
em1,em2=np.meshgrid(e,e)
emeshvo=(em1-em2)[nocc:,:nocc]
emeshf=emeshvo.flatten()
G_cphf=g_ijkl.copy()
G_cphf=4*G_cphf -G_cphf.swapaxes(3,1)-G_cphf.swapaxes(1,2)
G_cphf=G_cphf@C
G_cphf=(G_cphf.swapaxes(2,3)@C).swapaxes(3,2)
G_cphf=(G_cphf.swapaxes(1,3)@C).swapaxes(3,1)
G_cphf=(G_cphf.swapaxes(0,3)@C).swapaxes(3,0)
G_cphfvovo=G_cphf.copy()[nocc:,:nocc,nocc:,:nocc]    #aibj

In [288]:
U_0=dV_mo.copy()
for i in range(mol.nao):
    for a in range(mol.nao):
        if e[i] !=e[a] :
            U_0[i,a]/=(e[a]-e[i])

In [289]:
U_0vo=U_0[nocc:,:nocc]
U_0f=U_0vo.flatten()

In [290]:
dVvo=(dV_mo[nocc:,:nocc])
dVf=dVvo.flatten()

In [291]:
print(dVvo/emeshvo,'\n')
print(U_0vo)

[[0.09391977]] 

[[0.09391977]]


In [292]:
print(dVf/emeshf,'\n')
print(U_0f)

[0.09391977] 

[0.09391977]


In [293]:
def emul(_u):
    return _u/emeshf

In [294]:
emul(dVf)

array([0.09391977])

Now the inverse

In [295]:
print(dVf,'\n')
print(U_0f*emeshf)

[-0.0833446] 

[-0.0833446]


In [296]:
def emul2(_u):
    return _u*emeshf #from U gives dV

In [297]:
print(dVf,'\n')
print(emul2(U_0f))

[-0.0833446] 

[-0.0833446]


In [298]:
def emul3(_u):
    return _u*emeshf-_u #from U gives dV

In [299]:
uk=krylov(emul3,dVf)

In [300]:
print(U_0f)
print(uk)
np.allclose(uk,U_0f)

[0.09391977]
[0.09391977]


True

now add the electronic repulsion response other part !!!!

In [301]:
def gmul(_u):
    return np.einsum('ijkl,kl->ij',G_cphfvovo,_u.reshape(nvir,nocc)).flatten()

In [302]:
amuk(dVf)

array([0.00096938])

In [313]:
def amul(_u):
    return emul3(_u)+gmul(_u)/2

In [315]:
uk=krylov(amul,dVf)

In [316]:
print(uk)
print(U_0f)

[0.09330829]
[0.09391977]


In [317]:
test(U_0f)

0.0009032652882911935
0.3005681473646777
0.29966488207638653


In [318]:
test(uk)

0.0028543013674316854
0.3005681473646777
0.29771384599724604


In [307]:
dP_app=C@(U_0@O-O@U_0)@C.T    
print(np.linalg.norm(dP_app-dP))
print(np.linalg.norm(dP))
print(np.linalg.norm(dP_app))

0.0009032652882911935
0.3005681473646777
0.29966488207638653
